# Twilio SMS Getting Started Guide

## Finding and Purchasing an Available Number and Sending a Message

This guide will walk you through step-by-step getting started with Twilio SMS

# Walkthrough
First, we need to import os and Client from twilio.rest

In [4]:
import os
from twilio.rest import Client
import twilio

Next, you will want to create another file, info.py, in which you will store your account SID and auth token. Name the variables accountSID and authToken respectively. Then, import the file so that we can access these variables.

In [5]:
import info

We want to write a function that will create a client using these variables:

In [6]:
def create_client():
    try:
        account_sid = os.environ.get("ACCOUNT_SID", info.accountSID)
        auth_token = os.environ.get("AUTH_TOKEN", info.authToken)
        client = Client(account_sid, auth_token)
        return client
    except AttributeError:
        print("Make sure you correctly type your personal information in a file named 'info.py'.")
        return None

Using our client, we want to create a function that will search for available phone numbers so we may purchase one to use. I have chosen to search for available phone numbers by area code in the US with the area code '540'. You can change this to whatever area code you prefer.

This function with create a list **local** of 20 available phone numbers. 

After creating the list of local numbers, the function will pick the first one from the list, change its format to '+1xxxxxxxxxx', and return the number.

In [7]:
def get_number(client, area_code):
    try:
        limit = 20
        local = client.available_phone_numbers('US').local.list(
                                                                area_code,
                                                                limit
                                                            )
        number = local[0].friendly_name
        my_number = "+1" + number[1:4] + number[6:9] + number[10:14]
        return my_number
    except twilio.base.exceptions.TwilioException:
        return ("Your Twilio credentials were incorrect.")
    except IndexError:
        return("Area code entered was invalid.")

Now that I have my number, the following **purchase_number** function will purchase it on my account:

In [8]:
def purchase_number(client, my_number):
    try:
        incoming_phone_number = client.incoming_phone_numbers \
                                        .create(phone_number=my_number)
        return("Number has successfully been purchased.")
    except twilio.base.exceptions.TwilioRestException:
        return("Problem with twilio account.")

Finally, I can send my message with the function **send_message** below. This will send my message from the number I have just purchased to my personal number stored as a variable in my info.py file. You can change the body to say whatever you want. 

In [9]:
def send_message(client, my_number, personalNum):
    message = client.messages \
                    .create(
                         body="This is my message.",
                         from_= my_number,
                         to= personalNum
                     )

    return message.sid

After creating our functions we can run them. First we will create our client variable.

In [ ]:
client = create_client()

Next we'll use our **get_number** function to find a local number for us to purchase and print it. Before running, the console will ask us to input and area code to search for.

In [ ]:
area_code = input("Type area code to search for: ")
my_number = get_number(client, area_code)
print("This is my Twilio number: " + my_number)

The next cell will purchase the number we just found.

In [ ]:
print(purchase_number(client, my_number))

The last cell will send a message from the purchased number to you personal number.

In [ ]:
message_sid = send_message(client, my_number, info.personalNum)

# Testing Our Code

In [10]:
import unittest
import re

In [13]:
class TestNotebook(unittest.TestCase):
    def test_info_py_format(self):
        #Is info.personalNum in e. 164 format?
        self.assertTrue(re.match('^\+[1-9]\d{1,14}$', info.personalNum))
    def test_get_number_output_format(self):
        account_sid = os.environ.get("ACCOUNT_SID", info.accountSID)
        auth_token = os.environ.get("AUTH_TOKEN", info.authToken)
        client = Client(account_sid, auth_token)
        #Is get_number() output in e. 164 format?
        self.assertTrue(re.match('^\+[1-9]\d{1,14}$', get_number(client, '540')))
    def test_wrong_credentials(self):
        account_sid = os.environ.get("ACCOUNT_SID", '1')
        auth_token = os.environ.get("AUTH_TOKEN", '2')
        client = Client(account_sid, auth_token)
        self.assertEqual(get_number(client, '540'), "Your Twilio credentials were incorrect.")
    def test_wrong_area_code(self):
        account_sid = os.environ.get("ACCOUNT_SID", info.accountSID)
        auth_token = os.environ.get("AUTH_TOKEN", info.authToken)
        client = Client(account_sid, auth_token)
        self.assertEqual(get_number(client, '1'), "Area code entered was invalid.")

In [14]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_get_number_output_format (__main__.TestNotebook) ... ok
test_info_py_format (__main__.TestNotebook) ... ok
test_wrong_area_code (__main__.TestNotebook) ... ERROR
test_wrong_credentials (__main__.TestNotebook) ... ERROR

ERROR: test_wrong_area_code (__main__.TestNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-7-b4afd16f618b>", line 8, in get_number
    number = local[0].friendly_name
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-13-dc4a7f98608b>", line 20, in test_wrong_area_code
    self.assertEqual(get_number(client, '1'), "Area code entered was invalid.")
  File "<ipython-input-7-b4afd16f618b>", line 11, in get_number
    except twilio.base.exceptions.TwilioException:
NameError: name 'twilio' is not defined

ERROR: test_wrong_credentials (__main__.TestNotebook)
------------